In [1]:
# 导入必要的库
import collections
import math 
import random
import sys
import time
import os
import numpy as np

import tensorflow as tf

In [2]:
import pathlib  # 面向对象的文件操作库

path = pathlib.Path("./ptb")

## 1.处理数据集

PTB（Penn Tree Bank）是一个常用的小型语料库。它采样自《华尔街日报》的文章，包括训练集、验证集和测试集。
我们将在PTB训练集上训练词嵌入模型。该数据集的每一行作为一个句子。句子中的每个词由空格隔开。

In [3]:
# 查看所需数据集是否存在
assert 'ptb.train.txt' in os.listdir("./ptb")

In [4]:
with (path.joinpath("ptb.train.txt")).open() as f:
    # 按行读取所有
    lines = f.readlines()
    # 按句分词
    raw_dataset = [sentence.split() for sentence in lines]
# 打印长度，及分词样例    
print("sentences length: %d" % len(raw_dataset))
print(raw_dataset[:1])

sentences length: 42068
[['aer', 'banknote', 'berlitz', 'calloway', 'centrust', 'cluett', 'fromstein', 'gitano', 'guterman', 'hydro-quebec', 'ipo', 'kia', 'memotec', 'mlx', 'nahb', 'punts', 'rake', 'regatta', 'rubens', 'sim', 'snack-food', 'ssangyong', 'swapo', 'wachter']]


对于数据集的前3个句子，打印每个句子的词数和前5个词。这个数据集中句尾符为“”，生僻词全用“<UNK>”表示，数字则被替换成了“N”。

In [5]:
for raw in raw_dataset[:3]:
    print("tokens : ", len(raw), raw[:5])

tokens :  24 ['aer', 'banknote', 'berlitz', 'calloway', 'centrust']
tokens :  15 ['pierre', '<unk>', 'N', 'years', 'old']
tokens :  11 ['mr.', '<unk>', 'is', 'chairman', 'of']


## 2.建立词语索引

In [6]:
counter = collections.Counter([token for sentence in raw_dataset for token in sentence])
counter = dict(filter(lambda X:X[1], counter.items()))

然后将词语映射到整数索引

In [7]:
idx2token = [token for token, _ in counter.items()]
# word --> idx
token2idx = {token: idx for idx, token in enumerate(idx2token)}
# idx --> token
dataset = [[token2idx[token] for token in sentence] for sentence in raw_dataset]

In [8]:
dataset[:3]

[[0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23],
 [24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 26],
 [38, 25, 39, 40, 41, 25, 42, 31, 43, 44, 45]]

In [9]:
tokens = [len(sentence) for sentence in dataset]
tokens[:5]

[24, 15, 11, 23, 34]

In [10]:
num_tokens = sum(tokens)
num_tokens

887521

## 3.二次采样


文本数据中一般会出现一些高频词，如英文中的“the”“a”和“in”。通常来说，在一个背景窗口中，一个词（如“chip”）和较低频词（如“microprocessor”）同时出现比和较高频词（如“the”）同时出现对训练词嵌入模型更有益。因此，训练词嵌入模型时可以对词进行二次采样 [2]。
具体来说，数据集中每个被索引词$w_i$将有一定概率被丢弃，该丢弃概率为

$$ P(w_i) = \max\left(1 - \sqrt{\frac{t}{f(w_i)}}, 0\right),$$ 

其中 $f(w_i)$ 是数据集中词$w_i$的个数与总词数之比，常数$t$是一个超参数（实验中设为$10^{-4}$）。可见，只有当$f(w_i) > t$时，我们才有可能在二次采样中丢弃词$w_i$，并且越高频的词被丢弃的概率越大。

In [11]:
def discard(idx, t=1e-4): 
#     token = idx2token[idx]
# #     print(token)
#     counts = counter[token]
#     fw = counts / num_tokens
#     Pw = max(1-math.sqrt(t / counts), 0)
    return random.uniform(0, 1) < 1 - math.sqrt(
        1e-4 / counter[idx2token[idx]] * num_tokens)

subsample_dataset = [[token for token in sentence if not discard(token)] for sentence in dataset]

In [12]:
# 比较一个词在二次采样后次数的变化
def compare_counts(token):
    return '# %s: before=%d, after=%d' % (token, sum(
    [st.count(token2idx[token]) for st in dataset]), sum(
    [st.count(token2idx[token]) for st in subsample_dataset]))
# 常见词减少
compare_counts('the')

'# the: before=50770, after=2127'

In [13]:
# 但低频次完整保留
compare_counts('join')

'# join: before=45, after=45'

## 4.提取中心词和背景词

我们将与中心词距离不超过背景窗口大小的词作为它的背景词。下面定义函数提取出所有中心词和它们的背景词。它每次在整数1和`max_window_size`（最大背景窗口）之间随机均匀采样一个整数作为背景窗口大小。

In [14]:
def get_centers_and_contexts(dataset, max_window_size):
    centers, contexts = [], []
    for st in dataset:
        if len(st) < 2:  # 每个句子至少要有2个词才可能组成一对“中心词-背景词”
            continue
        centers += st
        for center_i in range(len(st)):
            window_size = random.randint(1, max_window_size)
            indices = list(range(max(0, center_i - window_size),
                                 min(len(st), center_i + 1 + window_size)))
            indices.remove(center_i)  # 将中心词排除在背景词之外
            contexts.append([st[idx] for idx in indices])
    return centers, contexts

创建一个人工数据集，其中含有词数分别为7和3的两个句子。设最大背景窗口为2，打印所有中心词和它们的背景词。

In [15]:
tiny_dataset = [list(range(7)), list(range(7, 10))]
print('dataset', tiny_dataset)
for center, context in zip(*get_centers_and_contexts(tiny_dataset, 2)):
    print('center', center, 'has contexts', context)

dataset [[0, 1, 2, 3, 4, 5, 6], [7, 8, 9]]
center 0 has contexts [1]
center 1 has contexts [0, 2, 3]
center 2 has contexts [0, 1, 3, 4]
center 3 has contexts [2, 4]
center 4 has contexts [2, 3, 5, 6]
center 5 has contexts [3, 4, 6]
center 6 has contexts [4, 5]
center 7 has contexts [8, 9]
center 8 has contexts [7, 9]
center 9 has contexts [7, 8]


实验中，设置最大背景窗口大小为5

In [17]:
all_centers, all_contexts = get_centers_and_contexts(subsample_dataset, 5)

In [21]:
all_contexts[:1]

[[1, 2, 3]]

## 5.负采样

我们使用负采样来进行近似训练。对于一对中心词和背景词，我们随机采样$K$个噪声词（实验中设$K=5$）。根据word2vec论文的建议，噪声词采样概率$P(w)$设为$w$词频与总词频之比的0.75次方 [2]。

In [18]:
def get_negatives(all_contexts, sampling_weights, K=5):
    all_negtives, neg_candidates, i = [], [], 0
    # 
    population = list(range(len(sampling_weights)))
    for contexts in all_contexts:
        negatives = []
        while len(negatives) < len(contexts) * K:
            if i == len(neg_candidates):
                # 根据每个词的权重（sampling_weights）随机生成k个词的索引作为噪声词。
                # 为了高效计算，可以将k设得稍大一点
                i, neg_candidates = 0, random.choices(
                    population, sampling_weights, k=int(1e5))
            neg, i = neg_candidates[i], i + 1
            # 噪声词不能是背景词
            if neg not in set(contexts):
                negatives.append(neg)
                
        all_negtives.append(negatives)
    return all_negtives

In [20]:
sampling_weights = [counter[w]**0.75 for w in idx2token]
all_negatives = get_negatives(all_contexts, sampling_weights, 5)

## 6.读取数据

我们从数据集中提取所有中心词`all_centers`，以及每个中心词对应的背景词`all_contexts`和噪声词`all_negatives`。我们将通过随机小批量来读取它们。

在一个小批量数据中，第$i$个样本包括一个中心词以及它所对应的$n_i$个背景词和$m_i$个噪声词。由于每个样本的背景窗口大小可能不一样，其中背景词与噪声词个数之和$n_i+m_i$也会不同。在构造小批量时，我们将每个样本的背景词和噪声词连结在一起，并添加填充项0直至连结后的长度相同，即长度均为$\max_i n_i+m_i$（`max_len`变量）。为了避免填充项对损失函数计算的影响，我们构造了掩码变量`masks`，其每一个元素分别与连结后的背景词和噪声词`contexts_negatives`中的元素一一对应。当`contexts_negatives`变量中的某个元素为填充项时，相同位置的掩码变量`masks`中的元素取0，否则取1。为了区分正类和负类，我们还需要将`contexts_negatives`变量中的背景词和噪声词区分开来。依据掩码变量的构造思路，我们只需创建与`contexts_negatives`变量形状相同的标签变量`labels`，并将与背景词（正类）对应的元素设1，其余清0。

下面我们实现这个小批量读取函数`batchify`。它的小批量输入`data`是一个长度为批量大小的列表，其中每个元素分别包含中心词`center`、背景词`context`和噪声词`negative`。该函数返回的小批量数据符合我们需要的格式，例如，包含了掩码变量。

In [33]:
def batchify(data):
    max_len = max(len(c) + len(n) for _, c, n in data)
    centers, contexts_negatives, masks, labels = [], [], [], []
    for center, context, negative in data:
        center=center.numpy().tolist()
        context=context.numpy().tolist()
        negative=negative.numpy().tolist()
        cur_len = len(context) + len(negative)
        centers += [center]
        contexts_negatives += [context + negative + [0] * (max_len - cur_len)]
        masks += [[1] * cur_len + [0] * (max_len - cur_len)]
        labels += [[1] * len(context) + [0] * (max_len - len(context))]
        
    return tf.data.Dataset.from_tensor_slices((tf.reshape(tf.convert_to_tensor(centers),shape=(-1, 1)), tf.convert_to_tensor(contexts_negatives),
            tf.convert_to_tensor(masks), tf.convert_to_tensor(labels)))  # return Dataset

定义的batchify函数指定DataLoader实例中小批量的读取方式，然后打印读取的第一个批量中各个变量的形状。

In [39]:
# 构造一个生成器函数
def generator():
    for cent, cont, neg in zip(all_centers, all_contexts, all_negatives):
        yield (cent, cont, neg)

In [40]:
# 设置批次处理大小
BATCH_SIZE = 512
# 设置缓冲区处理大小
BUFFER_SIZE = len(all_centers)
dataset = tf.data.Dataset.from_generator(generator=generator, output_types=(tf.int32, tf.int32, tf.int32))  # return Dataset
dataset = dataset.apply(batchify).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [37]:
len(all_centers)

374920

In [41]:
# 查看一个批次的数据维度
# for batch in dataset:  
#     for name, data in zip(['centers', 'contexts_negatives', 'masks',
#                            'labels'], batch):
#         print(name, 'shape:', data.shape)
#     break

for batch in dataset.take(1):
    for name, data in zip(['centers', 'contexts_negatives', 'masks',
                           'labels'], batch):
        print(name, 'shape:', data.shape)

centers shape: (512, 1)
contexts_negatives shape: (512, 10)
masks shape: (512, 10)
labels shape: (512, 10)


## 7.Skip-Grams模型

### 7.1嵌入层

获取词嵌入的层称为嵌入层，在Keras中可以通过创建`layers.Embedding`实例得到。嵌入层的权重是一个矩阵，其行数为词典大小（`input_dim`），列数为每个词向量的维度（`output_dim`）。我们设词典大小为20，词向量的维度为4。

In [65]:
embed = tf.keras.layers.Embedding(input_dim=20, output_dim=4)
embed.build(input_shape=(1, 20))
embed.get_weights(), embed.get_weights()[0].shape # 大 E Matrix

([array([[-0.01389253, -0.01471822, -0.03376635, -0.0218811 ],
         [ 0.0268763 , -0.00022911,  0.02033422, -0.03386219],
         [ 0.01655303,  0.03307791, -0.04610068, -0.0126529 ],
         [-0.00707158, -0.04399356, -0.02574978, -0.01557447],
         [-0.00054859,  0.04827062, -0.03226428, -0.03512539],
         [ 0.03776315, -0.04736323, -0.01607998, -0.02834105],
         [ 0.03347426,  0.00806312,  0.04817868,  0.0388451 ],
         [-0.04529743,  0.03087929,  0.02833769,  0.04581187],
         [-0.04933009,  0.02003794,  0.03978861,  0.01821828],
         [-0.00517409, -0.00753533,  0.01901368, -0.02309676],
         [ 0.02146   , -0.04762582, -0.03873235, -0.03721379],
         [-0.02608201,  0.03023119, -0.02927792, -0.04744846],
         [ 0.01347772,  0.02857014, -0.01639032,  0.02482169],
         [-0.03037801,  0.00055902, -0.02525632,  0.03369306],
         [-0.00591332,  0.04755424,  0.02174342, -0.04153781],
         [ 0.02202249, -0.04205309, -0.03466513,  0.032

嵌入层的输入为词的索引。输入一个词的索引$i$，嵌入层返回权重矩阵的第$i$行作为它的词向量。下面我们将形状为(2, 3)的索引输入进嵌入层，由于词向量的维度为4，我们得到形状为(2, 3, 4)的词向量。

In [44]:
x = tf.convert_to_tensor([[1, 2, 3], [4, 5, 6]], dtype=tf.float32)
embed(x)

<tf.Tensor: id=10123158, shape=(2, 3, 4), dtype=float32, numpy=
array([[[ 0.01444234, -0.01542183, -0.01209927,  0.01701441],
        [-0.04900263, -0.00953865, -0.02516942,  0.04692339],
        [ 0.0019304 ,  0.01080089,  0.03861311, -0.01190361]],

       [[-0.03597053, -0.03199013, -0.03058745, -0.00400792],
        [-0.02806777,  0.04499402,  0.01625714, -0.02638143],
        [ 0.02417671, -0.01159408,  0.03410666,  0.02534369]]],
      dtype=float32)>

### 7.2小批量乘法

我们可以使用小批量乘法运算`batch_dot`对两个小批量中的矩阵一一做乘法。假设第一个小批量中包含$n$个形状为$a\times b$的矩阵$\boldsymbol{X}_1, \ldots, \boldsymbol{X}_n$，第二个小批量中包含$n$个形状为$b\times c$的矩阵$\boldsymbol{Y}_1, \ldots, \boldsymbol{Y}_n$。这两个小批量的矩阵乘法输出为$n$个形状为$a\times c$的矩阵$\boldsymbol{X}_1\boldsymbol{Y}_1, \ldots, \boldsymbol{X}_n\boldsymbol{Y}_n$。因此，给定两个形状分别为($n$, $a$, $b$)和($n$, $b$, $c$)的`NDArray`，小批量乘法输出的形状为($n$, $a$, $c$)。

In [45]:
X = tf.ones((2, 1, 4))
Y = tf.ones((2, 4, 6))
tf.matmul(X, Y).shape

TensorShape([2, 1, 6])

### 7.3skip-grams模型的前向计算

在前向计算中，跳字模型的输入包含中心词索引`center`以及连结的背景词与噪声词索引`contexts_and_negatives`。其中`center`变量的形状为(批量大小, 1)，而`contexts_and_negatives`变量的形状为(批量大小, `max_len`)。这两个变量先通过词嵌入层分别由词索引变换为词向量，再通过小批量乘法得到形状为(批量大小, 1, `max_len`)的输出。输出中的每个元素是中心词向量与背景词向量或噪声词向量的内积。

In [46]:
def skip_gram(center, contexts_and_negatives, embed_v, embed_u):
    v = embed_v(center)
    u = embed_u(contexts_and_negatives)
    # 转置后才能满足矩阵乘法要求
    pred = v @ tf.transpose(u, perm=[0, 2, 1])
    return pred

### 8.训练模型

根据负采样中损失函数的定义，我们可以直接使用Keras的二元交叉熵损失函数BinaryCrossEntropyLoss。

In [48]:
class SigmoidBinaryCrossEntropyLoss(tf.keras.losses.Loss):
    
    def __init__(self):
        super(SigmoidBinaryCrossEntropyLoss, self).__init__()
        
        
    def __call__(self, inputs, targets, mask=None):
        #tensorflow中使用tf.nn.weighted_cross_entropy_with_logits设置mask并没有起到作用
        #直接与mask按元素相乘回实现当mask为0时不计损失的效果
        inputs = tf.cast(inputs, dtype=tf.float32)
        targets = tf.cast(targets, dtype=tf.float32)
        mask = tf.cast(mask, tf.float32)
        loss = tf.nn.sigmoid_cross_entropy_with_logits(targets, inputs) * mask
        return tf.reduce_mean(loss, axis=1)
    
loss = SigmoidBinaryCrossEntropyLoss()

可以通过掩码变量指定小批量中参与损失函数计算的部分预测值和标签：当掩码为1时，相应位置的预测值和标签将参与损失函数的计算；当掩码为0时，相应位置的预测值和标签则不参与损失函数的计算。我们之前提到，掩码变量可用于避免填充项对损失函数计算的影响。

In [49]:
pred = tf.convert_to_tensor([[1.5, 0.3, -1, 2], [1.1, -0.6, 2.2, 0.4]],dtype=tf.float32)
# 标签变量label中的1和0分别代表背景词和噪声词
label = tf.convert_to_tensor([[1, 0, 0, 0], [1, 1, 0, 0]],dtype=tf.float32)
mask = tf.convert_to_tensor([[1, 1, 1, 1], [1, 1, 1, 0]],dtype=tf.float32)  # 掩码变量
loss(label, pred, mask) * mask.shape[1] / tf.reduce_sum(mask,axis=1)

<tf.Tensor: id=10123186, shape=(2,), dtype=float32, numpy=array([0.47317582, 0.9398902 ], dtype=float32)>

作为比较，下面将从零开始实现二元交叉熵损失函数的计算，并根据掩码变量mask计算掩码为1的预测值和标签的损失。

In [50]:
def sigmoid(x):
    return - math.log(1 / (1 + math.exp(-x)))

print('%.4f' % ((sigmoid(1.5) + sigmoid(-0.3) + sigmoid(1) + sigmoid(-2)) / 4)) # 注意1-sigmoid(x) = sigmoid(-x)
print('%.4f' % ((sigmoid(1.1) + sigmoid(-0.6) + sigmoid(-2.2)) / 3))

0.8740
1.2100


### 8.1 初始化模型参数

我们分别构造中心词和背景词的嵌入层，并将超参数词向量维度`embed_size`设置成100

In [52]:
embed_size = 100
net = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(idx2token), output_dim=embed_size),
    tf.keras.layers.Embedding(input_dim=len(idx2token), output_dim=embed_size)
])
net.get_layer(index=0)

### 8.2定义训练函数

In [56]:
def train(net, lr, num_epochs):
    optimizer = tf.optimizers.Adam(lr)
    for epoch in range(num_epochs):
        start, l_sum, n = time.process_time(), 0.0, 0
        for batch in dataset:
            center, context_negative, mask, label = [d for d in batch]
            mask=tf.cast(mask,dtype=tf.float32)
            with tf.GradientTape(persistent=True) as tape:
                pred = skip_gram(center, context_negative, net.get_layer(index=0), net.get_layer(index=1))
                # 使用掩码变量mask来避免填充项对损失函数计算的影响
                l = (loss(label, tf.reshape(pred,label.shape), mask) *
                     mask.shape[1] / tf.reduce_sum(mask,axis=1))
                l=tf.reduce_mean(l)# 一个batch的平均loss
            grads = tape.gradient(l, net.variables)
            optimizer.apply_gradients(zip(grads, net.variables))
            l_sum += l.numpy().item()
            n += 1
        print('epoch %d, loss %.2f, time %.2fs'
              % (epoch + 1, l_sum / n, time.process_time() - start))

In [60]:
train(net, 0.003, 10)

epoch 1, loss -154070.48, time 222.11s
epoch 2, loss -161297.27, time 224.62s
epoch 3, loss -168719.80, time 226.92s
epoch 4, loss -176355.39, time 225.20s
epoch 5, loss -184185.38, time 227.12s
epoch 6, loss -192221.86, time 228.66s
epoch 7, loss -200388.98, time 252.42s
epoch 8, loss -208758.83, time 231.62s
epoch 9, loss -217328.43, time 223.12s
epoch 10, loss -226004.17, time 235.81s


### 8.3应用词嵌入模型

In [62]:
def get_similar_tokens(query_token, k, embed):
    W = embed.get_weights()
#     W = [np.array]
    W = tf.convert_to_tensor(W[0])
    x = W[token2idx[query_token]]
    x = tf.reshape(x,shape=[-1,1])
    # 添加的1e-9是为了数值稳定性， 防止除0
    cos = tf.reshape(tf.matmul(W, x),shape=[-1])/ tf.sqrt(tf.reduce_sum(W * W, axis=1) * tf.reduce_sum(x * x) + 1e-9)
#     cos = tf.reshape(tf.matmul(W, x),shape=[-1]) / (tf.linalg.norm(W) * tf.linalg.norm(x))
    _, topk = tf.math.top_k(cos, k=k+1)
    topk=topk.numpy().tolist()
    for i in topk[1:]:  # 除去输入词
        print('cosine sim=%.3f: %s' % (cos[i], (idx2token[i])))
        
get_similar_tokens('chip', 3, net.get_layer(index=0))

cosine sim=1.000: in
cosine sim=1.000: or
cosine sim=1.000: recent
